In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb


/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/unlabeled.csv')

sub_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [3]:
def weighted_classt(x): 
    if pd.isna(x):
        return np.nan
    elif x < 15: 
        return 'very severely underweight' 
    elif x >= 15 and x < 16: 
        return 'severely weight' 
    elif x >=16 and x < 18.5: 
        return 'underweight' 
    elif x >= 18.5 and x < 25: 
        return 'healthy weight' 
    elif x >= 25 and x < 30: 
        return 'overweight'
    elif x >= 30 and x < 35: 
        return 'class 1' 
    elif x >= 35 and x < 40: 
        return 'class 2' 
    else: 
        return 'class 3' 

In [4]:
train_data['weightclass'] = train_data['bmi'].map(weighted_classt)
test_data['weightclass'] = test_data['bmi'].map(weighted_classt)

In [5]:
cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string','integer'])]['Variable Name'])
cat_var.remove('icu_admit_type')
cat_var.remove('bmi')
cat_var = cat_var[1:]

In [6]:
#cat_var = cat_var[1:]
#cat_var = cat_var[1:]
cat_var = cat_var + ['weightclass']

In [7]:
cont_var = list(data_dict[data_dict['Data Type'].isin(['numeric'])]['Variable Name'])
cont_var.remove('pred')
cont_var = cont_var + ['bmi']

#### Baseline

In [8]:
x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [9]:
x_test = test_data.drop('hospital_death',axis=1)

In [10]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [11]:
### impute bmi values based on height and weight
count = 0
bmi_list = []
for i in tqdm(range(x_comb.shape[0])):
    
    if pd.isnull(x_comb["weight"].iloc[i]) or pd.isnull(x_comb["height"].iloc[i]):
        bmi_list.append(x_comb["bmi"].iloc[i])
        continue
    else:
        
        count += 1
        bmi_list.append(x_comb["weight"].iloc[i] / ((x_comb["height"].iloc[i]/100) ** 2))

100%|██████████| 131021/131021 [00:06<00:00, 19624.92it/s]


In [12]:
x_comb['bmi'] = bmi_list

In [13]:
x_comb["apache_4a_hospital_death_prob"] = x_comb["apache_4a_hospital_death_prob"].replace(-1, np.nan)
x_comb["apache_4a_icu_death_prob"] = x_comb["apache_4a_icu_death_prob"].replace(-1, np.nan)

In [14]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [15]:
for col in tqdm(new_cat_var):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

100%|██████████| 32/32 [00:03<00:00,  9.54it/s]


In [16]:
new_cont_var = list(set(cont_var).intersection(set(x_comb.columns)))

In [17]:
x_comb["age"].mean()

62.00996491901006

In [18]:
for col in tqdm(new_cont_var):
    x_comb[col] = x_comb[col].fillna(x_comb[col].mean())

100%|██████████| 153/153 [00:00<00:00, 586.49it/s]


In [19]:
count_var = []
for i,col in tqdm(enumerate(new_cont_var + new_cat_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])
    count_var.append(str(col)+'count')

185it [00:12, 14.63it/s]


In [20]:
age_cat = []

for i in tqdm(range(0,len(x_comb))):
    
    val = x_comb['age'].iloc[i]
    
    if val >= 15 and val <= 24: 
        age_cat.append('igen')
    
    elif val >= 25 and val <= 54: 
       age_cat.append('Prime_working_Age')
    
    elif val >= 55 and val <= 64: 
        age_cat.append('Mature_working_Age')
        
    else: 
        age_cat.append('Elderly_working_Age')


100%|██████████| 131021/131021 [00:01<00:00, 68076.01it/s]


In [21]:
x_comb['age_category'] = age_cat
le = LabelEncoder()
x_comb['age_category'] = le.fit_transform(x_comb['age_category'].astype(str))

In [22]:
#List to be biined -----> VITALS
to_binned_columns = list(data_dict[data_dict['Category'] == 'vitals']['Variable Name'].unique())

In [23]:
#inter_cols3
#x_comb['d1_heartrate_max_cat'] = 

binned_cols = []

for cols in tqdm(to_binned_columns) :
    x_comb[cols+'_binned'] = list(pd.cut(x_comb[cols], 10,labels=False))
    binned_cols.append(cols+'_binned')
    

100%|██████████| 52/52 [00:02<00:00, 24.02it/s]


In [24]:
x_comb['height'+'_binned'] = list(pd.cut(x_comb['height'], 8,labels=False))

In [25]:
#binned_cols

In [26]:
#train_data.columns[0:100]

list_imp_cols = ['icu_type','weightclass','age_category','gender','d1_heartrate_max_binned','height_binned']

inter_cols4 = []

for col1 in tqdm(list_imp_cols) :
    for col2 in list_imp_cols:
        if col1 !=col2 :
            x_comb[col1+col2] = x_comb[col1].astype(str) + '_' + x_comb[col2].astype(str)
            inter_cols4.append(col1+col2)
            

100%|██████████| 6/6 [00:05<00:00,  1.01it/s]


In [27]:
for cols in tqdm(inter_cols4):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

100%|██████████| 30/30 [00:01<00:00, 16.44it/s]


In [ ]:
#train_data.columns
# list_groups = list(data_dict['Category'].unique())
# list_groups = list_groups[1:]

# grp_ls = []

# for grp in tqdm(list_groups):
    
#     grp_ls.append(list(data_dict[data_dict['Category'] == grp]['Variable Name']))

In [ ]:
# inter_cols5 = []

# for i in tqdm(range(0,len(grp_ls))):
#     p = grp_ls[i]
#     cols = list(set(p).intersection(set(x_comb.columns)))
#     #str_cols = '_'.join(cols)
    
#     x_comb['grp_cols'+str(i)] = x_comb[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
#     inter_cols5.append('grp_cols'+str(i))


In [ ]:
# for cols in tqdm(inter_cols5):
#     le = LabelEncoder()
#     x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

In [28]:
#2way interaction
bias_cols = ['gender','age']

In [29]:
inter_cols6 = []

for i in tqdm(bias_cols):
    
    for j in binned_cols :
        
        x_comb[i+'_'+j] = x_comb[i].astype(str) + '_' + x_comb[j].astype(str)
        
        inter_cols6.append(i+'_'+j)

100%|██████████| 2/2 [00:17<00:00,  8.93s/it]


In [30]:
#Critical -3way interaction

# inter_cols6 = []

x_comb['age'+'gender'+'weighted_classt'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['weightclass'].astype(str)
x_comb['age'+'gender'+'ethnicity'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str)

inter_cols6 = inter_cols6 + ['age'+'gender'+'weighted_classt','age'+'gender'+'ethnicity']


In [31]:
#inter_cols6

In [32]:
for cols in tqdm(inter_cols6):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

100%|██████████| 106/106 [00:08<00:00, 25.54it/s]


In [ ]:
#final_cat_var = new_cat_var + count_var + ['age_category'] + inter_cols1 + inter_cols2 + inter_cols3 + inter_cols4

##### Target Encoding

In [33]:
temp_train = x_train.copy()
temp_train['hospital_death'] = train_data['hospital_death']

In [34]:
cat_count_var = []

for i,col in tqdm(enumerate(new_cat_var)):
    mean = temp_train['hospital_death'].mean()
    
    #Compute the number of values and the mean of each group
    agg = temp_train.groupby(col)['hospital_death'].agg(['count', 'mean'])
    counts = agg['count']
    means =  agg['mean']
    
    #Compute the "smoothed" means
    m= 3
    smooth = (counts * means + m * mean) / (counts + m)
    
    #Final_val
    x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
    cat_count_var.append(str(col)+'count_new')
    

32it [00:01, 25.86it/s]


In [35]:
x_comb.shape

(131021, 593)

In [36]:
# # Remove Features with more than 70 percent missing values
data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
data_missing = data_missing.index[data_missing > 0.9]

x_comb = x_comb.drop(columns = data_missing)

In [37]:
cat_count_var_new = list(set(cat_count_var).intersection(x_comb.columns))
#x_comb.isna().sum()

In [38]:
for col in tqdm(cat_count_var_new):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

100%|██████████| 21/21 [00:04<00:00,  4.42it/s]


In [39]:
aggs = {}

for col in tqdm(new_cat_var):
    aggs[col] = ['nunique','count']
    

100%|██████████| 32/32 [00:00<00:00, 69148.75it/s]


In [40]:
new_cat_var = list(set(new_cat_var).intersection(set(x_comb.columns)))

In [41]:
#new_cat_var

In [42]:
for col in tqdm(new_cat_var):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
                    left_on=[col], right_on=[col])

100%|██████████| 32/32 [06:03<00:00, 18.43s/it]


In [43]:
x_comb['hospital_icu_id'] = x_comb['hospital_id'].astype(str) + '_' +  x_comb['icu_id'].astype(str)

le = LabelEncoder()
x_comb['hospital_icu_id'] = le.fit_transform(x_comb['hospital_icu_id'].astype(str))

In [44]:
x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)

In [45]:
train_x = x_comb[0:91713]
test_x = x_comb[91713:]

In [46]:
x_comb.shape

(131021, 2565)

In [47]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
                                                      test_size=0.2,
                                                      random_state=0,
                                                      stratify = y_train)

In [48]:
#let us make the catboost model, use_best_model params will make the model prevent overfitting
model = CatBoostClassifier(iterations=3000, 
                           learning_rate=0.04, 
                           l2_leaf_reg= 3.5,  
                           depth= 6, 
                           rsm=0.98, 
                           loss_function= 'Logloss', 
                           eval_metric='AUC',
                           use_best_model=True,
                           random_seed=42)

In [50]:
list_final_cat = list(set(new_cat_var+binned_cols).intersection(x_comb.columns))

In [52]:
model.fit(x1_train,y1_train, cat_features= list_final_cat , eval_set=(x1_val,y1_val))


0:	test: 0.7502640	best: 0.7502640 (0)	total: 587ms	remaining: 29m 21s
1:	test: 0.7927255	best: 0.7927255 (1)	total: 1.15s	remaining: 28m 46s
2:	test: 0.8187914	best: 0.8187914 (2)	total: 1.74s	remaining: 29m 1s
3:	test: 0.8405647	best: 0.8405647 (3)	total: 2.3s	remaining: 28m 44s
4:	test: 0.8412790	best: 0.8412790 (4)	total: 2.57s	remaining: 25m 40s
5:	test: 0.8441080	best: 0.8441080 (5)	total: 3.09s	remaining: 25m 41s
6:	test: 0.8474086	best: 0.8474086 (6)	total: 3.57s	remaining: 25m 28s
7:	test: 0.8502105	best: 0.8502105 (7)	total: 4s	remaining: 24m 54s
8:	test: 0.8500751	best: 0.8502105 (7)	total: 4.47s	remaining: 24m 44s
9:	test: 0.8538927	best: 0.8538927 (9)	total: 5.03s	remaining: 25m 3s
10:	test: 0.8591698	best: 0.8591698 (10)	total: 5.58s	remaining: 25m 15s
11:	test: 0.8594028	best: 0.8594028 (11)	total: 6.12s	remaining: 25m 23s
12:	test: 0.8649497	best: 0.8649497 (12)	total: 6.64s	remaining: 25m 26s
13:	test: 0.8652529	best: 0.8652529 (13)	total: 7.17s	remaining: 25m 30s
14:	

113:	test: 0.8897376	best: 0.8897376 (113)	total: 59.4s	remaining: 25m 2s
114:	test: 0.8898071	best: 0.8898071 (114)	total: 59.8s	remaining: 25m 1s
115:	test: 0.8897947	best: 0.8898071 (114)	total: 1m	remaining: 25m
116:	test: 0.8898307	best: 0.8898307 (116)	total: 1m	remaining: 25m
117:	test: 0.8898357	best: 0.8898357 (117)	total: 1m 1s	remaining: 24m 58s
118:	test: 0.8898529	best: 0.8898529 (118)	total: 1m 1s	remaining: 24m 57s
119:	test: 0.8899048	best: 0.8899048 (119)	total: 1m 2s	remaining: 24m 56s
120:	test: 0.8899678	best: 0.8899678 (120)	total: 1m 2s	remaining: 24m 54s
121:	test: 0.8900826	best: 0.8900826 (121)	total: 1m 3s	remaining: 24m 54s
122:	test: 0.8901536	best: 0.8901536 (122)	total: 1m 3s	remaining: 24m 52s
123:	test: 0.8902275	best: 0.8902275 (123)	total: 1m 4s	remaining: 24m 51s
124:	test: 0.8902633	best: 0.8902633 (124)	total: 1m 4s	remaining: 24m 50s
125:	test: 0.8904238	best: 0.8904238 (125)	total: 1m 5s	remaining: 24m 49s
126:	test: 0.8904852	best: 0.8904852 (126

222:	test: 0.8951258	best: 0.8951258 (222)	total: 2m 11s	remaining: 27m 13s
223:	test: 0.8951905	best: 0.8951905 (223)	total: 2m 12s	remaining: 27m 20s
224:	test: 0.8951951	best: 0.8951951 (224)	total: 2m 14s	remaining: 27m 37s
225:	test: 0.8952139	best: 0.8952139 (225)	total: 2m 15s	remaining: 27m 38s
226:	test: 0.8952432	best: 0.8952432 (226)	total: 2m 15s	remaining: 27m 36s
227:	test: 0.8952709	best: 0.8952709 (227)	total: 2m 17s	remaining: 27m 48s
228:	test: 0.8952881	best: 0.8952881 (228)	total: 2m 18s	remaining: 27m 59s
229:	test: 0.8952881	best: 0.8952881 (229)	total: 2m 19s	remaining: 27m 58s
230:	test: 0.8953092	best: 0.8953092 (230)	total: 2m 19s	remaining: 27m 56s
231:	test: 0.8953353	best: 0.8953353 (231)	total: 2m 20s	remaining: 27m 56s
232:	test: 0.8953591	best: 0.8953591 (232)	total: 2m 21s	remaining: 27m 57s
233:	test: 0.8953765	best: 0.8953765 (233)	total: 2m 21s	remaining: 27m 57s
234:	test: 0.8954383	best: 0.8954383 (234)	total: 2m 22s	remaining: 27m 56s
235:	test: 0

331:	test: 0.8974187	best: 0.8974357 (329)	total: 3m 13s	remaining: 25m 51s
332:	test: 0.8974389	best: 0.8974389 (332)	total: 3m 13s	remaining: 25m 49s
333:	test: 0.8974614	best: 0.8974614 (333)	total: 3m 14s	remaining: 25m 48s
334:	test: 0.8974763	best: 0.8974763 (334)	total: 3m 14s	remaining: 25m 47s
335:	test: 0.8974795	best: 0.8974795 (335)	total: 3m 15s	remaining: 25m 46s
336:	test: 0.8974872	best: 0.8974872 (336)	total: 3m 15s	remaining: 25m 45s
337:	test: 0.8974812	best: 0.8974872 (336)	total: 3m 16s	remaining: 25m 44s
338:	test: 0.8975069	best: 0.8975069 (338)	total: 3m 16s	remaining: 25m 43s
339:	test: 0.8975223	best: 0.8975223 (339)	total: 3m 17s	remaining: 25m 42s
340:	test: 0.8975456	best: 0.8975456 (340)	total: 3m 17s	remaining: 25m 40s
341:	test: 0.8975602	best: 0.8975602 (341)	total: 3m 18s	remaining: 25m 39s
342:	test: 0.8975510	best: 0.8975602 (341)	total: 3m 18s	remaining: 25m 38s
343:	test: 0.8975569	best: 0.8975602 (341)	total: 3m 19s	remaining: 25m 36s
344:	test: 0

440:	test: 0.8989892	best: 0.8989892 (440)	total: 4m 10s	remaining: 24m 13s
441:	test: 0.8990140	best: 0.8990140 (441)	total: 4m 11s	remaining: 24m 12s
442:	test: 0.8990210	best: 0.8990210 (442)	total: 4m 11s	remaining: 24m 12s
443:	test: 0.8990262	best: 0.8990262 (443)	total: 4m 12s	remaining: 24m 12s
444:	test: 0.8990442	best: 0.8990442 (444)	total: 4m 12s	remaining: 24m 12s
445:	test: 0.8990626	best: 0.8990626 (445)	total: 4m 13s	remaining: 24m 11s
446:	test: 0.8990546	best: 0.8990626 (445)	total: 4m 14s	remaining: 24m 11s
447:	test: 0.8990672	best: 0.8990672 (447)	total: 4m 14s	remaining: 24m 11s
448:	test: 0.8990907	best: 0.8990907 (448)	total: 4m 15s	remaining: 24m 11s
449:	test: 0.8991223	best: 0.8991223 (449)	total: 4m 16s	remaining: 24m 11s
450:	test: 0.8991140	best: 0.8991223 (449)	total: 4m 16s	remaining: 24m 11s
451:	test: 0.8991213	best: 0.8991223 (449)	total: 4m 17s	remaining: 24m 11s
452:	test: 0.8991218	best: 0.8991223 (449)	total: 4m 18s	remaining: 24m 11s
453:	test: 0

...

2567:	test: 0.9048018	best: 0.9050207 (2348)	total: 24m 19s	remaining: 4m 5s
2568:	test: 0.9047973	best: 0.9050207 (2348)	total: 24m 20s	remaining: 4m 4s
2569:	test: 0.9047985	best: 0.9050207 (2348)	total: 24m 20s	remaining: 4m 4s
2570:	test: 0.9048127	best: 0.9050207 (2348)	total: 24m 21s	remaining: 4m 3s
2571:	test: 0.9048142	best: 0.9050207 (2348)	total: 24m 21s	remaining: 4m 3s
2572:	test: 0.9048158	best: 0.9050207 (2348)	total: 24m 22s	remaining: 4m 2s
2573:	test: 0.9048284	best: 0.9050207 (2348)	total: 24m 23s	remaining: 4m 2s
2574:	test: 0.9048266	best: 0.9050207 (2348)	total: 24m 23s	remaining: 4m 1s
2575:	test: 0.9048240	best: 0.9050207 (2348)	total: 24m 24s	remaining: 4m 1s
2576:	test: 0.9048309	best: 0.9050207 (2348)	total: 24m 24s	remaining: 4m
2577:	test: 0.9048291	best: 0.9050207 (2348)	total: 24m 25s	remaining: 3m 59s
2578:	test: 0.9048365	best: 0.9050207 (2348)	total: 24m 25s	remaining: 3m 59s
2579:	test: 0.9048387	best: 0.9050207 (2348)	total: 24m 26s	remaining: 3m 58s

2673:	test: 0.9049477	best: 0.9050207 (2348)	total: 25m 22s	remaining: 3m 5s
2674:	test: 0.9049543	best: 0.9050207 (2348)	total: 25m 22s	remaining: 3m 5s
2675:	test: 0.9049521	best: 0.9050207 (2348)	total: 25m 23s	remaining: 3m 4s
2676:	test: 0.9049535	best: 0.9050207 (2348)	total: 25m 24s	remaining: 3m 3s
2677:	test: 0.9049644	best: 0.9050207 (2348)	total: 25m 24s	remaining: 3m 3s
2678:	test: 0.9049669	best: 0.9050207 (2348)	total: 25m 25s	remaining: 3m 2s
2679:	test: 0.9049633	best: 0.9050207 (2348)	total: 25m 25s	remaining: 3m 2s
2680:	test: 0.9049524	best: 0.9050207 (2348)	total: 25m 26s	remaining: 3m 1s
2681:	test: 0.9049528	best: 0.9050207 (2348)	total: 25m 26s	remaining: 3m 1s
2682:	test: 0.9049549	best: 0.9050207 (2348)	total: 25m 27s	remaining: 3m
2683:	test: 0.9049516	best: 0.9050207 (2348)	total: 25m 28s	remaining: 2m 59s
2684:	test: 0.9049480	best: 0.9050207 (2348)	total: 25m 28s	remaining: 2m 59s
2685:	test: 0.9049426	best: 0.9050207 (2348)	total: 25m 29s	remaining: 2m 58s

2779:	test: 0.9049599	best: 0.9050207 (2348)	total: 26m 23s	remaining: 2m 5s
2780:	test: 0.9049584	best: 0.9050207 (2348)	total: 26m 23s	remaining: 2m 4s
2781:	test: 0.9049676	best: 0.9050207 (2348)	total: 26m 24s	remaining: 2m 4s
2782:	test: 0.9049720	best: 0.9050207 (2348)	total: 26m 24s	remaining: 2m 3s
2783:	test: 0.9049753	best: 0.9050207 (2348)	total: 26m 25s	remaining: 2m 3s
2784:	test: 0.9049794	best: 0.9050207 (2348)	total: 26m 25s	remaining: 2m 2s
2785:	test: 0.9049779	best: 0.9050207 (2348)	total: 26m 26s	remaining: 2m 1s
2786:	test: 0.9049757	best: 0.9050207 (2348)	total: 26m 26s	remaining: 2m 1s
2787:	test: 0.9049716	best: 0.9050207 (2348)	total: 26m 27s	remaining: 2m
2788:	test: 0.9049671	best: 0.9050207 (2348)	total: 26m 28s	remaining: 2m
2789:	test: 0.9049754	best: 0.9050207 (2348)	total: 26m 28s	remaining: 1m 59s
2790:	test: 0.9049748	best: 0.9050207 (2348)	total: 26m 29s	remaining: 1m 59s
2791:	test: 0.9049742	best: 0.9050207 (2348)	total: 26m 29s	remaining: 1m 58s
27

2885:	test: 0.9049746	best: 0.9050207 (2348)	total: 27m 23s	remaining: 1m 4s
2886:	test: 0.9049811	best: 0.9050207 (2348)	total: 27m 23s	remaining: 1m 4s
2887:	test: 0.9049848	best: 0.9050207 (2348)	total: 27m 24s	remaining: 1m 3s
2888:	test: 0.9049921	best: 0.9050207 (2348)	total: 27m 24s	remaining: 1m 3s
2889:	test: 0.9049942	best: 0.9050207 (2348)	total: 27m 25s	remaining: 1m 2s
2890:	test: 0.9049963	best: 0.9050207 (2348)	total: 27m 25s	remaining: 1m 2s
2891:	test: 0.9050010	best: 0.9050207 (2348)	total: 27m 26s	remaining: 1m 1s
2892:	test: 0.9050063	best: 0.9050207 (2348)	total: 27m 27s	remaining: 1m
2893:	test: 0.9050006	best: 0.9050207 (2348)	total: 27m 27s	remaining: 1m
2894:	test: 0.9050001	best: 0.9050207 (2348)	total: 27m 28s	remaining: 59.8s
2895:	test: 0.9050178	best: 0.9050207 (2348)	total: 27m 28s	remaining: 59.2s
2896:	test: 0.9050154	best: 0.9050207 (2348)	total: 27m 29s	remaining: 58.6s
2897:	test: 0.9050180	best: 0.9050207 (2348)	total: 27m 29s	remaining: 58.1s
2898:

2993:	test: 0.9051161	best: 0.9051209 (2951)	total: 28m 24s	remaining: 3.41s
2994:	test: 0.9051167	best: 0.9051209 (2951)	total: 28m 24s	remaining: 2.85s
2995:	test: 0.9051131	best: 0.9051209 (2951)	total: 28m 25s	remaining: 2.28s
2996:	test: 0.9051271	best: 0.9051271 (2996)	total: 28m 25s	remaining: 1.71s
2997:	test: 0.9051312	best: 0.9051312 (2997)	total: 28m 26s	remaining: 1.14s
2998:	test: 0.9051283	best: 0.9051312 (2997)	total: 28m 27s	remaining: 569ms
2999:	test: 0.9051383	best: 0.9051383 (2999)	total: 28m 27s	remaining: 0us

bestTest = 0.9051382756
bestIteration = 2999



In [53]:
#last let us make the submission,note that you have to make the pred to be int!
pred = model.predict_proba(test_x)
preds= pred[:,1]

In [ ]:
#pred = lgb_clf.predict(test_x)

In [54]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = preds

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [55]:
#data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub_8thFeb2020_cat.csv',index = False)

In [ ]:
# fig, ax = plt.subplots(figsize=(12,30))
# lgb.plot_importance(lgb_clf, max_num_features=180, height=0.8, ax=ax)
# ax.grid(False)
# plt.title("LightGBM - Feature Importance", fontsize=12)
# plt.show()

In [56]:
kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [57]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(train_x, y_train)):
    
    x_tr =  train_x.iloc[train_index]
    y_tr =  y_train[train_index]
    x_te =  train_x.iloc[valid_index]
    y_te =  y_train[valid_index]

    
    model.fit(x1_train,y1_train, cat_features= list_final_cat , eval_set=(x1_val,y1_val))

    
    num_models += 1
    
    predictions += model.predict_proba(test_x)[:,1]

0it [00:00, ?it/s]

0:	test: 0.7502640	best: 0.7502640 (0)	total: 461ms	remaining: 23m 2s
1:	test: 0.7927255	best: 0.7927255 (1)	total: 997ms	remaining: 24m 53s
2:	test: 0.8187914	best: 0.8187914 (2)	total: 1.53s	remaining: 25m 32s
3:	test: 0.8405647	best: 0.8405647 (3)	total: 2.13s	remaining: 26m 34s
4:	test: 0.8412790	best: 0.8412790 (4)	total: 2.42s	remaining: 24m 11s
5:	test: 0.8441080	best: 0.8441080 (5)	total: 3.06s	remaining: 25m 26s
6:	test: 0.8474086	best: 0.8474086 (6)	total: 3.6s	remaining: 25m 38s
7:	test: 0.8502105	best: 0.8502105 (7)	total: 4.03s	remaining: 25m 5s
8:	test: 0.8500751	best: 0.8502105 (7)	total: 4.51s	remaining: 24m 59s
9:	test: 0.8538927	best: 0.8538927 (9)	total: 5.11s	remaining: 25m 28s
10:	test: 0.8591698	best: 0.8591698 (10)	total: 5.7s	remaining: 25m 49s
11:	test: 0.8594028	best: 0.8594028 (11)	total: 6.2s	remaining: 25m 44s
12:	test: 0.8649497	best: 0.8649497 (12)	total: 6.7s	remaining: 25m 40s
13:	test: 0.8652529	best: 0.8652529 (13)	total: 7.32s	remaining: 26m
14:	test

113:	test: 0.8897376	best: 0.8897376 (113)	total: 57.3s	remaining: 24m 10s
114:	test: 0.8898071	best: 0.8898071 (114)	total: 57.8s	remaining: 24m 9s
115:	test: 0.8897947	best: 0.8898071 (114)	total: 58.3s	remaining: 24m 9s
116:	test: 0.8898307	best: 0.8898307 (116)	total: 58.8s	remaining: 24m 9s
117:	test: 0.8898357	best: 0.8898357 (117)	total: 59.3s	remaining: 24m 8s
118:	test: 0.8898529	best: 0.8898529 (118)	total: 59.8s	remaining: 24m 7s
119:	test: 0.8899048	best: 0.8899048 (119)	total: 1m	remaining: 24m 6s
120:	test: 0.8899678	best: 0.8899678 (120)	total: 1m	remaining: 24m 5s
121:	test: 0.8900826	best: 0.8900826 (121)	total: 1m 1s	remaining: 24m 4s
122:	test: 0.8901536	best: 0.8901536 (122)	total: 1m 1s	remaining: 24m 3s
123:	test: 0.8902275	best: 0.8902275 (123)	total: 1m 2s	remaining: 24m 3s
124:	test: 0.8902633	best: 0.8902633 (124)	total: 1m 2s	remaining: 24m 2s
125:	test: 0.8904238	best: 0.8904238 (125)	total: 1m 3s	remaining: 24m 2s
126:	test: 0.8904852	best: 0.8904852 (126)	

222:	test: 0.8951258	best: 0.8951258 (222)	total: 1m 51s	remaining: 23m 11s
223:	test: 0.8951905	best: 0.8951905 (223)	total: 1m 52s	remaining: 23m 11s
224:	test: 0.8951951	best: 0.8951951 (224)	total: 1m 52s	remaining: 23m 11s
225:	test: 0.8952139	best: 0.8952139 (225)	total: 1m 53s	remaining: 23m 11s
226:	test: 0.8952432	best: 0.8952432 (226)	total: 1m 53s	remaining: 23m 11s
227:	test: 0.8952709	best: 0.8952709 (227)	total: 1m 54s	remaining: 23m 11s
228:	test: 0.8952881	best: 0.8952881 (228)	total: 1m 54s	remaining: 23m 10s
229:	test: 0.8952881	best: 0.8952881 (229)	total: 1m 55s	remaining: 23m 10s
230:	test: 0.8953092	best: 0.8953092 (230)	total: 1m 55s	remaining: 23m 9s
231:	test: 0.8953353	best: 0.8953353 (231)	total: 1m 56s	remaining: 23m 8s
232:	test: 0.8953591	best: 0.8953591 (232)	total: 1m 56s	remaining: 23m 8s
233:	test: 0.8953765	best: 0.8953765 (233)	total: 1m 57s	remaining: 23m 7s
234:	test: 0.8954383	best: 0.8954383 (234)	total: 1m 57s	remaining: 23m 7s
235:	test: 0.8954

331:	test: 0.8974187	best: 0.8974357 (329)	total: 2m 46s	remaining: 22m 16s
332:	test: 0.8974389	best: 0.8974389 (332)	total: 2m 46s	remaining: 22m 16s
333:	test: 0.8974614	best: 0.8974614 (333)	total: 2m 47s	remaining: 22m 15s
334:	test: 0.8974763	best: 0.8974763 (334)	total: 2m 47s	remaining: 22m 15s
335:	test: 0.8974795	best: 0.8974795 (335)	total: 2m 48s	remaining: 22m 14s
336:	test: 0.8974872	best: 0.8974872 (336)	total: 2m 48s	remaining: 22m 14s
337:	test: 0.8974812	best: 0.8974872 (336)	total: 2m 49s	remaining: 22m 14s
338:	test: 0.8975069	best: 0.8975069 (338)	total: 2m 49s	remaining: 22m 13s
339:	test: 0.8975223	best: 0.8975223 (339)	total: 2m 50s	remaining: 22m 13s
340:	test: 0.8975456	best: 0.8975456 (340)	total: 2m 50s	remaining: 22m 12s
341:	test: 0.8975602	best: 0.8975602 (341)	total: 2m 51s	remaining: 22m 12s
342:	test: 0.8975510	best: 0.8975602 (341)	total: 2m 51s	remaining: 22m 11s
343:	test: 0.8975569	best: 0.8975602 (341)	total: 2m 52s	remaining: 22m 10s
344:	test: 0

...

2671:	test: 0.9049583	best: 0.9050207 (2348)	total: 27m 5s	remaining: 3m 19s
2672:	test: 0.9049458	best: 0.9050207 (2348)	total: 27m 5s	remaining: 3m 18s
2673:	test: 0.9049477	best: 0.9050207 (2348)	total: 27m 6s	remaining: 3m 18s
2674:	test: 0.9049543	best: 0.9050207 (2348)	total: 27m 7s	remaining: 3m 17s
2675:	test: 0.9049521	best: 0.9050207 (2348)	total: 27m 7s	remaining: 3m 17s
2676:	test: 0.9049535	best: 0.9050207 (2348)	total: 27m 8s	remaining: 3m 16s
2677:	test: 0.9049644	best: 0.9050207 (2348)	total: 27m 9s	remaining: 3m 15s
2678:	test: 0.9049669	best: 0.9050207 (2348)	total: 27m 9s	remaining: 3m 15s
2679:	test: 0.9049633	best: 0.9050207 (2348)	total: 27m 10s	remaining: 3m 14s
2680:	test: 0.9049524	best: 0.9050207 (2348)	total: 27m 11s	remaining: 3m 14s
2681:	test: 0.9049528	best: 0.9050207 (2348)	total: 27m 11s	remaining: 3m 13s
2682:	test: 0.9049549	best: 0.9050207 (2348)	total: 27m 12s	remaining: 3m 12s
2683:	test: 0.9049516	best: 0.9050207 (2348)	total: 27m 12s	remaining: 3

2777:	test: 0.9049648	best: 0.9050207 (2348)	total: 28m 10s	remaining: 2m 15s
2778:	test: 0.9049671	best: 0.9050207 (2348)	total: 28m 10s	remaining: 2m 14s
2779:	test: 0.9049599	best: 0.9050207 (2348)	total: 28m 11s	remaining: 2m 13s
2780:	test: 0.9049584	best: 0.9050207 (2348)	total: 28m 12s	remaining: 2m 13s
2781:	test: 0.9049676	best: 0.9050207 (2348)	total: 28m 12s	remaining: 2m 12s
2782:	test: 0.9049720	best: 0.9050207 (2348)	total: 28m 13s	remaining: 2m 12s
2783:	test: 0.9049753	best: 0.9050207 (2348)	total: 28m 13s	remaining: 2m 11s
2784:	test: 0.9049794	best: 0.9050207 (2348)	total: 28m 14s	remaining: 2m 10s
2785:	test: 0.9049779	best: 0.9050207 (2348)	total: 28m 15s	remaining: 2m 10s
2786:	test: 0.9049757	best: 0.9050207 (2348)	total: 28m 15s	remaining: 2m 9s
2787:	test: 0.9049716	best: 0.9050207 (2348)	total: 28m 16s	remaining: 2m 8s
2788:	test: 0.9049671	best: 0.9050207 (2348)	total: 28m 16s	remaining: 2m 8s
2789:	test: 0.9049754	best: 0.9050207 (2348)	total: 28m 17s	remaini

2883:	test: 0.9050020	best: 0.9050207 (2348)	total: 29m 15s	remaining: 1m 10s
2884:	test: 0.9049756	best: 0.9050207 (2348)	total: 29m 15s	remaining: 1m 9s
2885:	test: 0.9049746	best: 0.9050207 (2348)	total: 29m 16s	remaining: 1m 9s
2886:	test: 0.9049811	best: 0.9050207 (2348)	total: 29m 17s	remaining: 1m 8s
2887:	test: 0.9049848	best: 0.9050207 (2348)	total: 29m 17s	remaining: 1m 8s
2888:	test: 0.9049921	best: 0.9050207 (2348)	total: 29m 18s	remaining: 1m 7s
2889:	test: 0.9049942	best: 0.9050207 (2348)	total: 29m 19s	remaining: 1m 6s
2890:	test: 0.9049963	best: 0.9050207 (2348)	total: 29m 19s	remaining: 1m 6s
2891:	test: 0.9050010	best: 0.9050207 (2348)	total: 29m 20s	remaining: 1m 5s
2892:	test: 0.9050063	best: 0.9050207 (2348)	total: 29m 20s	remaining: 1m 5s
2893:	test: 0.9050006	best: 0.9050207 (2348)	total: 29m 21s	remaining: 1m 4s
2894:	test: 0.9050001	best: 0.9050207 (2348)	total: 29m 22s	remaining: 1m 3s
2895:	test: 0.9050178	best: 0.9050207 (2348)	total: 29m 22s	remaining: 1m 3

2991:	test: 0.9051005	best: 0.9051209 (2951)	total: 30m 21s	remaining: 4.87s
2992:	test: 0.9051043	best: 0.9051209 (2951)	total: 30m 21s	remaining: 4.26s
2993:	test: 0.9051161	best: 0.9051209 (2951)	total: 30m 22s	remaining: 3.65s
2994:	test: 0.9051167	best: 0.9051209 (2951)	total: 30m 23s	remaining: 3.04s
2995:	test: 0.9051131	best: 0.9051209 (2951)	total: 30m 23s	remaining: 2.44s
2996:	test: 0.9051271	best: 0.9051271 (2996)	total: 30m 24s	remaining: 1.83s
2997:	test: 0.9051312	best: 0.9051312 (2997)	total: 30m 25s	remaining: 1.22s
2998:	test: 0.9051283	best: 0.9051312 (2997)	total: 30m 25s	remaining: 609ms
2999:	test: 0.9051383	best: 0.9051383 (2999)	total: 30m 26s	remaining: 0us

bestTest = 0.9051382756
bestIteration = 2999



5it [2:54:33, 2149.88s/it]


In [58]:
predictions_5 = predictions / 5.0

In [59]:
data_sub = x_test[['encounter_id']]

In [60]:
data_sub['hospital_death'] = predictions_5

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub_9thFeb2020_catb.csv',index = False)

In [ ]:
# a1 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble.csv')
# a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble_gm.csv')
# a3 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/Submission/sub_ensemble_gm1.csv')
# a4 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_ensemble_gm2.csv')
# a5 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub8_lgb_10fold1.csv')
# a6 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub8_lgb_10fold.csv')
# a7 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub10_lgb_kfold2.csv')
# a8 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub5_lgb_ensemble12_fold_01022020.csv')
# a9 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub11_ensemble_imp1.csv')
# a10 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub12_catb.csv')
a11 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub3_lgb_ensemble1_01022020.csv')
a12 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub4_ensemble1_01022020.csv')
a13 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub5_lgb_ensemble12_fold_01022020.csv')
a14 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub3_10fold_4thFeb2020.csv')
a15 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_ensemble_gm3.csv')
a16 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub6_10fold_4thFeb2020.csv')
a17 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub60_10fold_4thFeb2020.csv')
a18 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub2_ensemble_5thFeb2020.csv')
a19 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_5thFeb2020_type2.csv')

In [ ]:
sub = pd.DataFrame()
sub['encounter_id'] = a18['encounter_id']

In [ ]:
pred_ens = (
#             a1['hospital_death'] + a2['hospital_death'] + a3['hospital_death'] + 
#             a4['hospital_death'] + a5['hospital_death'] + a6['hospital_death'] +
#             a7['hospital_death'] + a8['hospital_death'] + a9['hospital_death'] +
#             a10['hospital_death'] + 
            a11['hospital_death'] + a12['hospital_death'] +
            a13['hospital_death'] + a14['hospital_death'] + a15['hospital_death'] +
            a16['hospital_death'] + a17['hospital_death'] + a18['hospital_death'] +
            a19['hospital_death']
            
           )/9.0
            

In [ ]:
sub['hospital_death'] = pred_ens

In [ ]:
sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub_type1_type2_ensemble_5thFeb2020.csv',index = False)

In [ ]:
#train_x['encounter_id'].unique().shape


In [ ]:
#last let us make the submission,note that you have to make the pred to be int!
pred = model.predict_proba(test_x)
preds= pred[:,1]

In [ ]:
sub_data

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = preds

In [ ]:
x_test.head()

In [ ]:
ls

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub2.csv',index = False)

In [ ]:
data_sub.shape